In [ ]:
import sys
!{sys.executable} -m pip install skyfield pandas numpy matplotlib astropy

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 1. Configuração do Caminho (Path) para encontrar a pasta 'src'
# Obtém o diretório atual onde o notebook está (pasta 'notebooks')
current_dir = os.getcwd()
# Sobe um nível para achar a raiz do projeto 'space-simulation'
project_root = os.path.abspath(os.path.join(current_dir, '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Diretório raiz adicionado: {project_root}")

# 2. Importações do seu Projeto
# ATENÇÃO: Agora importamos 'load_tles_smart' em vez de 'load_tles_from_url'
from src.data_handler import load_tles_smart, satellites_to_dataframe
from src.simulation import run_multi_object_simulation_and_validate
from src.constants import RADIUS_EARTH

print("Módulos carregados com sucesso.")

In [ ]:
# 1. Carrega todos os satélites ativos usando CACHE INTELIGENTE
# max_days=1.0 significa que se o arquivo tiver menos de 1 dia, ele não baixa de novo.
ts, all_satellites = load_tles_smart(max_days=1.0)

if not all_satellites:
    print("Erro crítico: Falha ao carregar satélites. Verifique a conexão ou o arquivo local.")
else:
    # 2. Converte para DataFrame
    df_satellites = satellites_to_dataframe(all_satellites)

    # 3. APLICAÇÃO DO FILTRO
    FILTER_NAME = 'STARLINK'
    MAX_SATELLITES = 50 # 50 é um bom número para validação rápida

    satellites_to_simulate_df = df_satellites[
        df_satellites['name'].str.contains(FILTER_NAME, case=False)
    ].head(MAX_SATELLITES)

    # Lista final de objetos Skyfield para o motor RK4
    list_of_objects = satellites_to_simulate_df['object'].tolist()

    print(f"\nTotal de TLEs carregados: {len(all_satellites)}")
    print(f"Satélites selecionados para simulação ({FILTER_NAME}): {len(list_of_objects)}")

In [ ]:
print("\n--- INICIANDO SIMULAÇÃO E VALIDAÇÃO MÚLTIPLA ---")
print(f"Processando {len(list_of_objects)} objetos. Isso pode levar alguns segundos...")

# Roda o motor de simulação (RK4 + J2 vs SGP4)
final_trajectories_df = run_multi_object_simulation_and_validate(ts, list_of_objects)

print("Simulação Múltipla concluída com sucesso!")
display(final_trajectories_df.head())

In [ ]:
# 4A: Tabela de Validação de Erro

if 'error_km' in final_trajectories_df.columns:
    # Remove as duplicatas de erro (o erro é o mesmo para todos os passos de um satélite)
    validation_summary = final_trajectories_df[['satellite_name', 'error_km']].drop_duplicates()

    print("\n--- RESUMO DE VALIDAÇÃO (ERRO DE POSIÇÃO FINAL APÓS 1H) ---")
    # Mostra os 5 piores erros primeiro (para discussão dos outliers)
    print(validation_summary.sort_values(by='error_km', ascending=False).head(10))

    # Estatísticas gerais
    avg_error = validation_summary['error_km'].mean()
    max_error = validation_summary['error_km'].max()
    print(f"\nERRO MÉDIO GERAL: {avg_error:.4f} km")
    print(f"PIOR ERRO (OUTLIER): {max_error:.4f} km")
else:
    print("Erro: Coluna 'error_km' não encontrada. Verifique src/simulation.py")

In [ ]:
# 4B: Visualização 3D do Tráfego

fig = plt.figure(figsize=(10, 10)) # Tamanho bom para projetor
ax = fig.add_subplot(111, projection='3d')

print("Gerando gráfico 3D...")

# Itera sobre os satélites no DataFrame e desenha as trajetórias
for name, group in final_trajectories_df.groupby('satellite_name'):
    # Desenha a trajetória
    ax.plot(group['rx'], group['ry'], group['rz'], label=name, alpha=0.5, linewidth=0.8)
    
    # Marca a posição final com um ponto
    ax.scatter(group['rx'].iloc[-1], group['ry'].iloc[-1], group['rz'].iloc[-1], marker='o', s=10)

# Desenho da Terra (Esfera)
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x_earth = RADIUS_EARTH * np.outer(np.cos(u), np.sin(v))
y_earth = RADIUS_EARTH * np.outer(np.sin(u), np.sin(v))
z_earth = RADIUS_EARTH * np.outer(np.ones(np.size(u)), np.cos(v))

ax.plot_surface(x_earth, y_earth, z_earth, color='blue', alpha=0.1) # Terra azulada e transparente

ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Z (km)')
ax.set_title(f'Simulação de Tráfego: {len(list_of_objects)} Satélites\n(Modelo RK4 + J2)')

# Ajuste de eixos para a Terra não ficar "achatada" visualmente
limit = RADIUS_EARTH + 2000
ax.set_xlim(-limit, limit)
ax.set_ylim(-limit, limit)
ax.set_zlim(-limit, limit)

plt.show()